In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

import pandas as pd
import json
from shapely.geometry import Polygon
import folium
from src.functions import functions_for_project as ffp
from src.data import data_dictionary as data_d
import numpy as np

In [104]:
df = pd.read_csv(
    'http://www.data4apurpose.com/client_html/full_processed_data.csv')

msoa_uk = ffp.load_geojson(
    'http://www.data4apurpose.com/client_html'
    '/Middle_Layer_Super_Output_Areas_December_2011_Boundaries_EW_BFC_ultra_simple.geojson'
)

data_diction = data_d.EV_britain().description

central_london = [
    'City of London', 'Camden', 'Greenwich', 'Hackney',
    'Hammersmith and Fulham', 'Islington', 'Kensington and Chelsea', 'Lambeth',
    'Lewisham', 'Southwark', 'Tower Hamlets', 'Wandsworth', 'Westminster'
]

greater_london = [
    'Croydon', 'Sutton', 'Brent', 'Ealing', 'Hounslow', 'Richmond', 'Kingston',
    'Merton', 'Bromley', 'Bexley', 'Havering', 'Barking and Dagenham',
    'Redbridge', 'Newham', 'Waltham Forest', 'Haringey', 'Enfield', 'Barnet',
    'Harrow', 'Hillingdon'
]

lad = [{'label': area, 'value': area} for area in df.lad13nm.unique()]

In [105]:
df = pd.merge(df, msoa_uk, left_on='msoa11cd', right_on='properties.msoa11cd')

In [106]:
x_rand = np.random.randint(1,61,60)
y_rand = np.random.randint(1,61,60)

In [110]:
app = dash.Dash('__name__')
server = app.server

app.layout = html.Div([
    html.Div([
        html.H1('Correlations'),
        dcc.Graph(id='corr_graph'),
        html.Div([dcc.RadioItems(id='log_yscale',options=[{'label':'Logarithmic y-axis','value':'log'},
                                               {'label':'Linear y-axis scale','value':'linear'}],value='linear'),
        dcc.RadioItems(id='log_xscale',options=[{'label':'Logarithmic x-axis','value':'log'},
                                               {'label':'Linear x-axis scale','value':'linear'}],value='linear')]),
        html.H3('Feature'),
        dcc.Dropdown(id='var1',
                     value='total_netafterhsing',
                     options=[{
                         'label': x,
                         'value': x
                     } for x in df.columns[4:]]),
        html.Div(id='text1', children='...waiting'),
        html.H3('Target Variable'),
        dcc.Dropdown(id='var2',
                     value='2019_q2',
                     options=[{
                         'label': x,
                         'value': x
                     } for x in df.columns[4:]]),
        html.Div(id='text2', children='...waiting')
    ],
        style={
        'width': '50%',
                 'display': 'inline-block'
    }),
    html.Div([
        html.H1("Mapping"),
        html.Iframe(id='map',
                    srcDoc=open('search.html', 'r').read(),
                    width='100%',
                    height='500px'),
        dcc.Dropdown(
            id='local_a', value=central_london+greater_london, options=lad, multi=True)
    ],
        style={
        'width': '50%',
                 'float': 'right',
                 'display': 'inline-block'
    })
])

In [111]:
@app.callback(Output('text1', 'children'), [Input('var1', 'value')])
def definitions(value):
    return data_diction[value]


@app.callback(Output('text2', 'children'), [Input('var2', 'value')])
def definitions(value):
    return data_diction[value]


@app.callback(Output('map', 'srcDoc'),
              [Input('local_a', 'value'),
               Input('var2', 'value')])
def remap(area, target):
    ffp.plot_area(df, target, *area)
    # print(*area,target)
    return open('search.html', 'r').read()


@app.callback(Output('corr_graph', 'figure'), [
    Input('var1', 'value'),
    Input('var2', 'value'),
    Input('local_a', 'value'),
    Input('log_yscale', 'value'),
    Input('log_xscale', 'value')
])
def plot_corr(x_value, y_value, ladname, yscale, xscale):
    # print(df[value2])
    return go.Figure(data=[
        go.Scatter(x=df[x_value],
                   y=df[y_value],
                   mode='markers',
                   name='Entire UK',
                   opacity=0.7,
                   marker=go.scatter.Marker(color='red',
                                            size=4,
                                            line=dict(width=0.5,
                                                      color='black'))),
        go.Scatter(
            x=df[df['lad13nm'].str.contains('|'.join(ladname))][x_value],
            y=df[df['lad13nm'].str.contains('|'.join(ladname))][y_value],
            text="ddd",
            name='Local Authorities Selected',
            mode='markers',
            opacity=1,
            marker=go.scatter.Marker(color='green',
                                     size=6,
                                     line=dict(width=0.5, color='black')))
    ],
                     layout=go.Layout(title='Correlation between variables',
                                      xaxis={
                                          'type': xscale,
                                          'title': x_value
                                      },
                                      yaxis={
                                          'type': yscale,
                                          'title': y_value
                                      },
                                      margin={
                                          'l': 40,
                                          'b': 30,
                                          't': 20,
                                          'r': 20
                                      },
                                      legend={
                                          'x': 0,
                                          'y': 1
                                      },
                                      hovermode='closest'))

In [ ]:
if __name__ == "__main__":
    # webbrowser.open('http://127.0.0.1:8050/', new=0, autoraise=True)
    app.run_server(debug=False)

 * Serving Flask app "__name__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Mar/2020 22:49:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2020 22:49:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2020 22:49:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2020 22:49:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2020 22:49:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2020 22:49:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2020 22:49:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2020 22:50:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2020 22:50:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2020 22:50:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2020 22:50:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2020 22:50:16] "POST /_dash-update-component

In [ ]:
# scaler = StandardScaler()
# chrg_select = pd.DataFrame(ppf.ColumnSelector('charge_points').fit_transform(df).values.reshape(-1,1))
# df['charge_points'] = ppf.FeatureLogTransform().fit_transform(chrg_select)

# print(data_d.EV_britain().description["income_score"])
# print(plot_area(merged_df,"crime_score","Southwark"))